# SET GDRIVE

In [5]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [9]:
!ls drive

My Drive


# DOWNLOAD FILE

In [10]:
!rm processed_data.zip
!curl -L -o processed_data.zip https://www.dropbox.com/s/5lmwlnejhd70jfv/processed_data.zip?dl=1
!rm -rf processed_data
!unzip processed_data.zip

rm: cannot remove 'processed_data.zip': No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  296M  100  296M    0     0  18.5M      0  0:00:16  0:00:16 --:--:-- 32.7M
Archive:  processed_data.zip
   creating: processed_data/
   creating: processed_data/data_dev/
  inflating: processed_data/data_dev/00E15BB2-5568-4466-BA18-A8A8D34FC61C.gui  
  inflating: processed_data/data_dev/00E15BB2-5568-4466-BA18-A8A8D34FC61C.png  
  inflating: processed_data/data_dev/03806667-47A8-4965-8E93-21E205D5E8A7.gui  
  inflating: processed_data/data_dev/03806667-47A8-4965-8E93-21E205D5E8A7.png  
  inflating: processed_data/data_dev/03F9E9A6-621A-4183-810B-3DF3E13A0F99.gui  
  inflating: processed_data/data_dev/03F9E9

In [11]:
# remove filenya dulu biar ga duplicate
!rm bootstrap.vocab
# download folder bahan training
!curl -O https://raw.githubusercontent.com/andrewsoohwanlee/pix2code-pytorch/master/bootstrap.vocab

rm: cannot remove 'bootstrap.vocab': No such file or directory
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   124  100   124    0     0    124      0  0:00:01 --:--:--  0:00:01   320


# IMPORT AND INSTALL TORCH

In [12]:
!pip3 install torch torchvision

    100% |████████████████████████████████| 519.5MB 34kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x59fd6000 @  0x7fc1ea6941c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
    100% |████████████████████████████████| 61kB 21.5MB/s 
    100% |████████████████████████████████| 2.0MB 4.5MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [0]:
import numpy as np
import pdb
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, models, transforms
from torch.nn.utils.rnn import pack_padded_sequence
from torch.autograd import Variable

# PARAMS 

In [0]:
# Hyperparams
batch_size = 4
embed_size = 256
num_epochs = 1000
learning_rate = 0.001
hidden_size = 512
num_layers = 1

# Other params
shuffle = True
num_workers = 2 

# Logging Variables
save_after_x_epochs = 50
log_step = 5

# Paths
data_dir = './processed_data/data_train/' 
model_path = './drive/My Drive/model/'
vocab_path = './bootstrap.vocab'

# Building Vocabulary

In [0]:
def load_doc(filename):
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

In [0]:
class Vocabulary(object):
    def __init__(self):
        self.word2idx = {}
        self.idx2word = {}
        self.idx = 0
        
    def add_word(self, word):
        if not word in self.word2idx:
            self.word2idx[word] = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1
    
    def __call__(self, word):
        if not word in self.word2idx:
            return self.word2idx['<unk>']
        return self.word2idx[word]
    
    def __len__(self):
        return len(self.word2idx)

In [0]:
def build_vocab(vocab_file_path):
    vocab = Vocabulary()
    
    # Load the vocab file (super basic split())
    words_raw = load_doc(vocab_file_path)
    words = set(words_raw.split(' '))
    
    for i, word in enumerate(words):
        vocab.add_word(word)
        
    vocab.add_word(' ')
    vocab.add_word('<unk>') # If we find and unknown word
    
    print('Created vocabulary of ' + str(len(vocab)) + ' items from ' + vocab_file_path)
    
    return vocab

In [6]:
# Load vocabulary
vocab = build_vocab(vocab_path)

vocab_size = len(vocab)

Created vocabulary of 19 items from ./bootstrap.vocab


# Build Dataset (images and captions)

In [0]:
class ImageHTMLDataSet(Dataset):
    def __init__(self, data_dir, vocab, transform=None):
        self.data_dir = data_dir
        self.vocab = vocab
        self.transform = transform
        
        self.raw_image_names = []
        self.raw_captions = []
        
        # Fetch all files from our data directory
        self.filenames = os.listdir(data_dir)
        self.filenames.sort()
        
        # Sort files based on their filetype
        # Assume associated training examples have same filenames
        for filename in self.filenames:
            # AMBIL 3 DIGIT TRAKHIR DARI NAMA FILENYA
            if filename[-3:] == 'png':
                # Store image filename
                self.raw_image_names.append(filename)
            elif filename[-3:] == 'gui':
                data = load_doc(data_dir + filename)
                self.raw_captions.append(data)
                
        print('Created dataset of ' + str(len(self)) + ' items from ' + data_dir)
        
    def __getitem__(self, idx):
        img_path = self.raw_image_names[idx]
        raw_caption = self.raw_captions[idx]
        # Get image from filesystem
        image = Image.open(os.path.join(self.data_dir, img_path)).convert('RGB')
        if self.transform is not None:
            image = self.transform(image)
        
        # Convert caption (string) to list of vocab ID's
        caption = []
        caption.append(self.vocab('<START>'))
        
        # Remove newlines, separate words with spaces
        tokens = ' '.join(raw_caption.split())
        
        # Add space before each comma
        tokens = tokens.replace(',', ' ,')
        
        # Split into words
        tokens = tokens.split(' ')
        
        caption.extend([self.vocab(token) for token in tokens])
        caption.append(self.vocab('<END>'))
        
        target = torch.Tensor(caption)
        
        return image, target
      
    def __len__(self):
        return len(self.raw_image_names)

In [0]:
  def collate_fn(data):
      # sort datalist by caption length; descending order
      data.sort(key = lambda data_pair: len(data_pair[1]), reverse=True)
      images, captions = zip(*data)

      # Merge images ( from tuple of 3D Tensor to 4D Tensor)
      images = torch.stack(images, 0)

      # Merge captions ( from tuple of 1D tensor to 2D tensor)
      lengths = [len(caption) for caption in captions] # List of caption lengths
      targets = torch.zeros(len(captions), max(lengths)).long()

      for i, caption in enumerate(captions):
          end = lengths[i]
          targets[i, :end] = caption[:end]

      return images, targets, lengths

In [9]:
# Image preprocessing, normalization for the pretrained resnet
transform = transforms.Compose([ 
    transforms.Resize((224,224)),
    transforms.ToTensor(), 
    transforms.Normalize((0.485, 0.456, 0.406), 
                         (0.229, 0.224, 0.225))])

# Create data loader
img_html_dataset = ImageHTMLDataSet(data_dir=data_dir, vocab=vocab, transform=transform)
data_loader = DataLoader(dataset=img_html_dataset,
                         batch_size=batch_size,
                         shuffle=shuffle,
                         num_workers=num_workers,
                         collate_fn=collate_fn)

Created dataset of 1400 items from ./processed_data/data_train/


# ENCODER MODEL

In [0]:
class EncoderCNN(nn.Module):
    def __init__(self, embed_size):
        super(EncoderCNN, self).__init__()
        resnet = models.resnet152(pretrained = True)
        modules = list(resnet.children())[:-1]
        self.resnet = nn.Sequential(*modules)
        self.linear = nn.Linear(resnet.fc.in_features, embed_size)
        self.bn = nn.BatchNorm1d(embed_size, momentum=0.01)
        
        
    def forward(self, images):
        with torch.no_grad():
          features = self.resnet(images)
        features = features.reshape(features.size(0), -1)        
        features = self.bn(self.linear(features))        
        return features

# DECODER MODEL

In [0]:
class DecoderRNN(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size, num_layers, max_seq_length=100):
        super(DecoderRNN, self).__init__()
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size,hidden_size, num_layers, batch_first=True)        
        self.linear = nn.Linear(hidden_size, vocab_size)
        self.max_seg_length = max_seq_length
    
    def forward(self, features, captions, lengths): 
        embeddings = self.embed(captions)
        embeddings = torch.cat((features.unsqueeze(1), embeddings), 1)
        packed = pack_padded_sequence(embeddings, lengths, batch_first=True)
        hiddens, _ = self.lstm(packed)
        outputs = self.linear(hiddens[0])
        return outputs
      
    def sample(self, features, states=None):
        sampled_ids = []
        inputs = features.unsqueeze(1)      
        for i in range(self.max_seg_length):
            hiddens, states = self.lstm(inputs, states)
            outputs = self.linear(hiddens.squeeze(1))
            _, predicted = outputs.max(1)
            sampled_ids.append(predicted)
            inputs = self.embed(predicted)
            inputs = inputs.unsqueeze(1)
        sampled_ids = torch.stack(sampled_ids, 1)
        return sampled_ids

# Training

In [0]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [0]:
# BUILD MODELNYA
encoder = EncoderCNN(256).to(device)
decoder = DecoderRNN(256, 512, len(vocab), 1).to(device)
#  LOSS AND OPTIMIZER
criterion = nn.CrossEntropyLoss()
params = list(decoder.parameters()) + list(encoder.linear.parameters()) + list(encoder.bn.parameters())
optimizer = torch.optim.Adam(params, lr=0.001)  

In [14]:
total_step = len(data_loader)
total_step

350

In [0]:
def save_checkpoint(state):
    torch.save(state, os.path.join(model_path, 'checkpoint-{}-{}.pth'.format(epoch+1, i+1)))

In [0]:
for epoch in range(300):
    for i, (images, captions, lengths) in enumerate(data_loader):
        
        # Set mini-batch dataset
        images = images.to(device)
        captions = captions.to(device)
        targets = pack_padded_sequence(captions, lengths, batch_first=True)[0]
        
        # Forward, backward and optimize
        features = encoder(images)
        outputs = decoder(features, captions, lengths)        
        loss = criterion(outputs, targets)
        encoder.zero_grad()
        decoder.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 50 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss:{:.4f}, Perplexity: {:5.4f}'
                  .format(epoch, 300, i+1, total_step, loss.item(), np.exp(loss.item())))
                  
        # Save the model checkpoints
        if (epoch+1) % 100 == 0 and (i+1) % total_step == 0:
            save_checkpoint({
              'epoch': epoch + 1,
              'encoder': encoder.state_dict(),
              'decoder': decoder.state_dict(),
              'optimizer' : optimizer.state_dict(),
            })

Epoch [0/300], Step [50/350], Loss:0.3188, Perplexity: 1.3755
Epoch [0/300], Step [100/350], Loss:0.2432, Perplexity: 1.2754
Epoch [0/300], Step [150/350], Loss:0.2217, Perplexity: 1.2482
Epoch [0/300], Step [200/350], Loss:0.2081, Perplexity: 1.2314
Epoch [0/300], Step [250/350], Loss:0.2180, Perplexity: 1.2436
Epoch [0/300], Step [300/350], Loss:0.1896, Perplexity: 1.2087
Epoch [0/300], Step [350/350], Loss:0.2162, Perplexity: 1.2414
Epoch [1/300], Step [50/350], Loss:0.1886, Perplexity: 1.2076
Epoch [1/300], Step [100/350], Loss:0.1838, Perplexity: 1.2017
Epoch [1/300], Step [150/350], Loss:0.1954, Perplexity: 1.2158
Epoch [1/300], Step [200/350], Loss:0.1673, Perplexity: 1.1821
Epoch [1/300], Step [250/350], Loss:0.1756, Perplexity: 1.1919
Epoch [1/300], Step [300/350], Loss:0.1787, Perplexity: 1.1956
Epoch [1/300], Step [350/350], Loss:0.1693, Perplexity: 1.1845
Epoch [2/300], Step [50/350], Loss:0.1561, Perplexity: 1.1689
Epoch [2/300], Step [100/350], Loss:0.1633, Perplexity: 1.